In [1]:
import pandas as pd
import numpy as np

mdf = pd.read_csv('data/df_movieID_title.csv')

user_list = [('Notebook, The ', '1'),
             ('Road to Perdition ', '2'),
             ('Karate Kid, The ', '3'),
             ('Pulp Fiction ', '4'),
             ('Zoolander ', '4'),
             ('Majestic, The ', '5'),
             ('Notorious ', '2'),
             ('Monty Python and the Holy Grail ', '2'),
             ('Apocalypse Now ', '1')]

id_list = []
for movie in user_list:
    idx = mdf[mdf.title_only == movie[0]].index.tolist()[0]
    id_list.append(mdf.iloc[idx]['movieId'])

id_list_string = [str(x) for x in id_list]

rating_list = []
for rating in user_list:
    rat = rating[1]
    rating_list.append(rat)

rating_list_float = [float(x) for x in rating_list]

user_input_df = pd.DataFrame(list(zip(id_list_string, rating_list_float)), columns =['movieId', 'rating'])
user_input_df['userId'] = '999999'
user_input_df = user_input_df[['userId','movieId', 'rating']]
#user_input_df

In [2]:
from surprise import SVD, KNNBasic, Reader, Dataset, accuracy, dump

reader = Reader()
new_data = Dataset.load_from_df(user_input_df, reader)

In [3]:
import pandas as pd

from surprise import SVD, KNNBasic, Reader, Dataset, accuracy, dump
from surprise.model_selection import train_test_split

ratings = pd.read_csv('data/ratings.csv')
#wide = ratings.pivot(index = 'userId', columns = 'movieId', values = 'rating')
ratings = ratings [['userId', 'movieId', 'rating']]
concat_df = pd.concat([ratings, user_input_df], axis = 0, ignore_index = True)
concat_df.head()

new_row = {'userId':['999999', '999999', '999999', '999999', '999999', '999999','999999', '999999', '999999', '999999', '999999', '999999'],
           'movieId':['3','70', '367', '200', '190', '1792', '8999', '10922', '6763', '13729', '34532', '89222'],
           'rating':[3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5]}

new_row_df = pd.DataFrame.from_dict(new_row)

# new_row1 = {'userId':'999999', 'movieId':'3', 'rating':0}
# new_row2 = {'userId':'999999', 'movieId':'70', 'rating':0}
# new_row3 = {'userId':'999999', 'movieId':'367', 'rating':0}

#append row to the dataframe
concat_df = concat_df.append(new_row_df, ignore_index=True)

In [4]:
concat_df.tail()

,userId,movieId,rating
100852,999999,10922,3.5
100853,999999,6763,3.5
100854,999999,13729,3.5
100855,999999,34532,3.5
100856,999999,89222,3.5


In [5]:
reader = Reader()
new_data = Dataset.load_from_df(concat_df, reader)

In [6]:
trainset = new_data.build_full_trainset()

# Build an algorithm, and train it.
# algo = KNNBasic()
# algo.fit(trainset)

algo = SVD(n_factors = 23, random_state = 666)
algo.fit(trainset)

uid = str(999999)  # raw user id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
# pred = algo.predict(uid, iid, r_ui=1, verbose=True)

In [8]:
iid = concat_df['movieId'].to_list()
iid_list_string = [str(x) for x in iid]

r_uid_list = concat_df['rating'].to_list()

In [10]:
predictions_list = []

for item in iid_list_string[-20:]:
    for r in r_uid_list[-20:]:
        predictions_list.append(algo.predict(uid, item, r_ui = r, verbose=True))

user: 999999     item: 5464       r_ui = 2.00   est = 3.05   {'was_impossible': False}
user: 999999     item: 5464       r_ui = 3.00   est = 3.05   {'was_impossible': False}
user: 999999     item: 5464       r_ui = 4.00   est = 3.05   {'was_impossible': False}
user: 999999     item: 5464       r_ui = 4.00   est = 3.05   {'was_impossible': False}
user: 999999     item: 5464       r_ui = 5.00   est = 3.05   {'was_impossible': False}
user: 999999     item: 5464       r_ui = 2.00   est = 3.05   {'was_impossible': False}
user: 999999     item: 5464       r_ui = 2.00   est = 3.05   {'was_impossible': False}
user: 999999     item: 5464       r_ui = 1.00   est = 3.05   {'was_impossible': False}
user: 999999     item: 5464       r_ui = 3.50   est = 3.05   {'was_impossible': False}
user: 999999     item: 5464       r_ui = 3.50   est = 3.05   {'was_impossible': False}
user: 999999     item: 5464       r_ui = 3.50   est = 3.05   {'was_impossible': False}
user: 999999     item: 5464       r_ui = 3.